In [17]:
%load_ext autoreload
%autoreload 2
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/aremath/.local/lib/python3.10/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['Infinity']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [18]:
import sys
sys.path.append("..")

In [19]:
import functools

In [20]:
from encoding.parse_rooms import parse_rooms, parse_exits, dictify_rooms

In [21]:
rooms = parse_rooms("../encoding/dsl/rooms_for_alloy.txt")
exits = parse_exits("../encoding/dsl/exits_for_alloy.txt")
design = dictify_rooms(rooms, exits)

In [22]:
len(design["Rooms"])

255

In [23]:
from rom_tools.rom_manager import RomManager
from abstraction_validation.sm_paths import *

In [24]:
from bdds.bdd_core import *

In [25]:
from functools import reduce

In [26]:
all_nodes = []
for r, room in rooms.items():
    for node in room.graph.name_node.keys():
        all_nodes.append(node)
node_ids = {n:i for i,n in enumerate(all_nodes)}

In [27]:
len(node_ids)

698

In [28]:
# Issue with design translation via chopping off the last _
node_ids["Spore_Spawn_Spawn"] = node_ids["Spore_Spawn_Spore_Spawn"]
node_ids["Golden_Torizo_Torizo"] = node_ids["Golden_Torizo_Golden_Torizo"]
node_ids["Mother_Brain_Brain"] = node_ids["Mother_Brain_Mother_Brain"]

In [29]:
def combine_bdd_group(bdds):
    return reduce(lambda x,y: x|y, bdds, context_id.false)

In [30]:
def loc_id(room_name, node_name, when="prev"):
    node_id = node_ids[f"{room_name}_{node_name}"]
    return f"node_id_{when} = {node_id}"

In [31]:
@functools.cache
def item_transitions(item_gained=None):
    if item_gained is None:
        return "(items_unchanged)"
    clauses = []
    for i in design["Items"] | design["Bosses"]:
        if i == item_gained:
            clause = f"{i}_prev < {i}_next"
        else:
            clause = f"{i}_prev = {i}_next"
        clauses.append(clause)
    return " & ".join(clauses)

def itemset_to_str(itemset):
    if len(itemset) == 0:
        return "TRUE"
    else:
        return " & ".join([f"{item}_prev = 1" for item in itemset])

def required_itemsets(itemsets):
    return "(" + " | ".join([itemset_to_str(itemset) for itemset in itemsets]) + ")"

In [32]:
from bdds.node_bdds import *

In [33]:
context_id = mk_context_id(node_ids)

In [34]:
len(context_id.bdd.vars) / 4

46.0

In [35]:
# Build individual BDDs
room_bdds = []
door_bdds = []
for room_name, room in tqdm(design['Rooms'].items()):
    links = []
    for node_name in room['Nodes']:
        s = loc_id(room_name, node_name) + " & " + loc_id(room_name, node_name, when="next") + " & " + item_transitions()
        links.append(context_id.add_expr(s, with_ops=True))
        if node_name in room['Drops']:
            s = loc_id(room_name, node_name) + " & " + loc_id(room_name, node_name, when="next") + " & " + item_transitions(room['Drops'][node_name])
            links.append(context_id.add_expr(s, with_ops=True))
    for node_name, door in room['Doors'].items():
        d = loc_id(room_name, node_name) + " & " + loc_id(door['Room'], door['Node'], when="next") + " & " + item_transitions()
        door_bdds.append(context_id.add_expr(d, with_ops=True))
    for node_name, edges in room['Edges'].items():
        for edge in edges:
            other_node_name = edge['Terminal']
            s = loc_id(room_name, node_name) + " & " + loc_id(room_name, other_node_name, when="next") + " & " + required_itemsets(edge['Requirements']) + " & " + item_transitions()
            links.append(context_id.add_expr(s, with_ops=True))
    room_bdd = combine_bdd_group(links)
    #print(room_bdd.count())
    room_bdds.append(room_bdd)

100%|█████████████████████████████████████████████████████████████████████████████████| 255/255 [00:12<00:00, 21.24it/s]


In [36]:
doors_bdd = combine_bdd_group(door_bdds)
rooms_bdd = combine_bdd_group(room_bdds)
all_bdd = doors_bdd | rooms_bdd

In [37]:
all_bdd.dag_size # 11K

15347

In [40]:
len(all_bdd.support)

92

In [41]:
context_id.bdd.dump("../output/all_bdd.json", [all_bdd])

In [42]:
all_bdd.count()
# 158,211,025,928,192

158658239397888.0

In [43]:
all_bdd.__dir__()

['__new__',
 '__repr__',
 '__hash__',
 '__str__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__invert__',
 '__and__',
 '__rand__',
 '__or__',
 '__ror__',
 '__int__',
 '__len__',
 'implies',
 'equiv',
 'let',
 'exist',
 'forall',
 'pick',
 'count',
 '_index',
 'var',
 'level',
 'ref',
 'low',
 'high',
 'negated',
 'support',
 'dag_size',
 'bdd',
 '_ref',
 '__doc__',
 '__pyx_vtable__',
 '__reduce__',
 '__setstate__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__init__',
 '__reduce_ex__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [44]:
a = context_id.exist(nexts, all_bdd)

In [45]:
a.count() * 2 ** len(set(prevs) - context_id.support(a))

23983097380864.0

In [46]:
context_id.count(all_bdd)

158658239397888

In [47]:
158211025928192 / 47897475284992.0

3.303118274748924

In [48]:
context_id.bdd.count(context_id.exist(nexts, all_bdd), len(prevs))

93683974144.0

In [49]:
from data_types import item_set

In [50]:
def mk_itemset_expr(itemset, when="prev"):
    clauses = []
    for i in item_mapping:
        if i in itemset:
            clause = f"{i}_{when} = 1"
        else:
            clause = f"{i}_{when} = 0"
        clauses.append(clause)
    return " & ".join(clauses)

In [51]:
def get_reachable_id(trans_norule, context, start_bdd):
    n = 0
    covered_p = start_bdd
    covered_last_p = context.false
    while covered_p != covered_last_p:
        covered_last_p = covered_p
        fringe_n = context.exist(prevs, covered_p & trans_norule)
        covered_p |= context.let(next_to_prev, fringe_n)
        print(n, covered_p.dag_size)
        n+=1
    return covered_p

In [32]:
start_bdd = context_id.add_expr("node_id_prev = 0 & " + mk_itemset_expr(item_set.ItemSet()))

In [34]:
%%capture
all_reachable_p = get_reachable_id(all_bdd, context_id, start_bdd)

In [38]:
feasible_trans = all_bdd & all_reachable_p

In [39]:
wincon_p = context_id.add_expr("node_id_prev = 2 & Mother_Brain_prev = 1")

In [40]:
wincon = context_id.add_expr("node_id_goal = 2 & Mother_Brain_goal = 1 & node_id_next = 2 & Mother_Brain_next = 1")

In [41]:
feasible_trans.count()

48891250.0

In [42]:
import json

In [43]:
#with open("../output/context_id_order.json", "r") as f:
#    context_id.bdd.reorder(json.load(f))
#context_id.bdd.configure(reordering=False)

In [50]:
#wincon = context_id.add_expr("node_id_next = node_id_goal & Mother_Brain_next = Mother_Brain_goal")
#wincon = context_id.add_expr("node_id_next = node_id_goal & node_id_goal = 1") # ~1.5s
wincon = context_id.add_expr("node_id_next = node_id_goal")

In [49]:
%%time
#TODOs
# Goal BDD x
#   More restrictive goal_bdd using pick w/ transitive closure
#   Forward & backward feasibility restrictions
# ~covered_prev_goal earlier
# Shattergate
all_policy = mk_policy_id2(feasible_trans, context_id, wincon)
#all_policy = mk_policy_id2(all_bdd, context_id, wincon)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112


KeyboardInterrupt: 

In [79]:
"node_id_next_0".rsplit("_", 2)

['node_id', 'next', '0']

In [80]:
all_policy_det = determinize2(all_policy, context_id)

In [81]:
all_policy.count()

8593935525.0

In [82]:
all_policy.dag_size

236455

In [83]:
all_policy_det.dag_size

234782

In [52]:
len(all_policy.support)

102

In [84]:
#task_expr = context_id.add_expr("Mother_Brain_prev < Mother_Brain_goal & node_id_prev = 2 & node_id_goal = 2")
task_expr = context_id.add_expr("node_id_prev = 0 & node_id_goal = 1")
# Concrete task
task = context_id.pick(context_id.exist(nexts, all_policy & task_expr))
task_bdd = context_id.assign_from(task)

In [85]:
controller_pn = context_id.exist(goals, all_policy_det & task_bdd)

In [97]:
%%time
edges = get_edges(all_policy_det, task_bdd, context_id)

CPU times: user 55.2 ms, sys: 180 µs, total: 55.4 ms
Wall time: 55.2 ms


In [96]:
%%time
path = get_path_concrete_id(all_policy, task_bdd, context_id)

CPU times: user 297 ms, sys: 0 ns, total: 297 ms
Wall time: 296 ms


In [88]:
%%time
path = get_path_concrete_id(all_policy, task_bdd, context_id)

CPU times: user 298 ms, sys: 0 ns, total: 298 ms
Wall time: 297 ms


In [89]:
steps = get_steps_id2(all_policy_det, task_bdd, context_id)

In [90]:
steps.dag_size

8450

In [94]:
%%time
#TODO: redo this notebook in the context of node_bdds.py (remove the code that I already moved)
#TODO: replace the existing path advice infrastructure with this data structure
#TODO: add the "end of game" location walled by MB, then fully determinize path
sorted(edges)

CPU times: user 44 µs, sys: 2 µs, total: 46 µs
Wall time: 46.7 µs


[(0, 0, 3),
 (1, 3, 12),
 (2, 12, 11),
 (3, 11, 54),
 (4, 54, 57),
 (5, 57, 59),
 (6, 59, 60),
 (7, 60, 62),
 (8, 62, 63),
 (9, 63, 148),
 (10, 148, 149),
 (11, 149, 149),
 (12, 149, 147),
 (13, 147, 151),
 (14, 151, 152),
 (15, 152, 174),
 (16, 174, 175),
 (17, 175, 175),
 (18, 175, 174),
 (19, 174, 152),
 (20, 152, 151),
 (21, 151, 147),
 (22, 147, 148),
 (23, 148, 63),
 (24, 63, 62),
 (25, 62, 60),
 (26, 60, 59),
 (27, 59, 57),
 (28, 57, 54),
 (29, 54, 11),
 (30, 11, 13),
 (31, 13, 66),
 (32, 66, 67),
 (33, 67, 64),
 (34, 64, 65),
 (35, 65, 65),
 (36, 65, 64),
 (37, 64, 67),
 (38, 67, 66),
 (39, 66, 13),
 (40, 13, 8),
 (41, 8, 71),
 (42, 71, 70),
 (43, 70, 79),
 (44, 79, 81),
 (45, 81, 76),
 (46, 76, 75),
 (47, 75, 73),
 (48, 73, 74),
 (49, 74, 91),
 (50, 91, 99),
 (51, 99, 119),
 (52, 119, 121),
 (53, 121, 123),
 (54, 123, 133),
 (55, 133, 133),
 (56, 133, 127),
 (57, 127, 134),
 (58, 134, 135),
 (59, 135, 136),
 (60, 136, 138),
 (61, 138, 137),
 (62, 137, 102),
 (63, 102, 104),
 (

In [ ]:
len(edges)

In [ ]:
len(path)

In [99]:
#context_id.bdd.dump("../output/node_policy_det.dddmp", [all_policy_det])

In [75]:
context_id.exist(nexts + prevs, all_policy).count

1.0

In [56]:
with open("../output/context_id_order.json", "w") as f:
    json.dump(context_id.bdd.var_levels, f)

In [43]:
all_policy = context_id.bdd.load("../output/policy_id.json")[0]

In [44]:
#context_id.bdd.dump("../output/policy_id.dddmp", [all_policy])

In [34]:
goal_satisfied_id = mk_goal_satisfied_id(context_id)

In [35]:
rev_node_ids = {v:k for k,v in node_ids.items()}

In [36]:
# task_bdd is built by first creating a task expr, then existing off the nexts, then picking and cubing it.
def get_path_concrete_id(policy, task_bdd, context):
    # Get a concrete path
    current = context.pick(task_bdd)
    path = [current]
    n = 0
    while True:
        #print(n, current["node_id_prev"])
        #print(n, rev_node_ids[current["node_id_prev"]])
        #print(current["node_id_prev"])
        #print(current)
        n += 1
        #current_bdd = context.bdd.cube(current)
        current_bdd = context.assign_from(current)
        next_temp = context.exist(prevs, current_bdd & policy)
        advice = context.let(next_to_prev, next_temp)
        #print(advice.count())
        current = context.pick(advice)
        # Check whether the goal is satisfied in the state given by the advice.
        if (next_temp & goal_satisfied_id).pick():
            break
        path.append(current)
    return path

# is cube() faster?
def get_path_concrete_id2(policy, task_bdd, context):
    # Get a concrete path
    current = task_bdd.pick()
    path = [current]
    while True:
        #print(current)
        current_bdd = context.bdd.cube(current)
        next_temp = context.exist(prevs, current_bdd & policy)
        advice = context.let(next_to_prev, next_temp)
        #print(advice.count())
        current = advice.pick()
        # Check whether the goal is satisfied in the state given by the advice.
        if (next_temp & goal_satisfied_id).pick():
            break
        path.append(current)
    return path

def get_first_step(policy, task_dict, context):
    # Get a concrete path
    task_bdd = context.assign_from(task_dict)
    return context.pick(task_bdd & policy)

In [37]:
#task_expr = context_id.add_expr("Mother_Brain_prev < Mother_Brain_goal & node_id_prev = 2 & node_id_goal = 2")
task_expr = context_id.add_expr("M_prev < M_goal & node_id_prev = 2 & node_id_goal = 2")
# Concrete task
task = context_id.pick(context_id.exist(nexts, all_policy & task_expr))
task_bdd = context_id.assign_from(task)

In [38]:
#cc = context_id.exist(prevs, all_bdd & context_id.add_expr("node_id_prev = 2"))
#cc = context_id.exist(prevs, all_bdd & task_bdd)
t2 = task.copy()
t2["node_id_prev"] = node_ids["Landing_Site_Ship"]
t2_bdd = context_id.assign_from(t2)
cc = context_id.exist(prevs, all_policy & t2_bdd)

In [39]:
nonidns = [k for k in context_id.vars.keys() if k != "node_id_next"]

In [40]:
for i in context_id.pick_iter(context_id.exist(nonidns, cc)):
    print(rev_node_ids[i["node_id_next"]])

Landing_Site_L2


In [41]:
# 1.94s concrete1
# 0.233s concrete2

In [42]:
%%time
x = get_path_concrete_id2(all_policy, t2_bdd, context_id)

CPU times: user 476 ms, sys: 14 µs, total: 476 ms
Wall time: 476 ms


In [45]:
%%time
x = get_path_concrete_id2(all_policy, task_bdd, context_id)

CPU times: user 486 ms, sys: 32 µs, total: 486 ms
Wall time: 486 ms


In [46]:
len(x)

567

In [1]:
%%time
step = get_first_step(all_policy, task, context_id)

NameError: name 'get_first_step' is not defined

In [262]:
%%time
context_id.assign_from(task)
#task_bdd & all_policy

CPU times: user 1.89 ms, sys: 103 µs, total: 2 ms
Wall time: 2.01 ms


Function (DdNode) with var index: 4, ref count: 2, int repr: 480166083

In [236]:
len(x)

2